In [1]:
import pickle
import pandas as pd
import numpy as np
from importlib import reload
from helpers import constants; reload(constants)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from helpers.helper_functions import LossAndErrorPrintingCallback
from tensorflow.keras.callbacks import CSVLogger
from datetime import datetime
import os
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.initializers import Constant

In [2]:
select_label = constants.SELECT_LABEL
intermediate_path = constants.ITM_DATA_DIR
model_data_path = constants.PRCD_DATA_DIR
sample_data_path = constants.SAMPLE_DATA_DIR
max_len = constants.MAX_SEQUENCE_LENGTH # max number of words in a post to use
max_word_no = constants.MAX_NUM_WORDS # how many unique words to use (i.e num rows in embedding vector)
embedding_path = constants.GLOVE_DIR
embedding_dim = constants.EMBEDDING_DIM # the number of element for one word in Glove Embedding
tokenizer_path = constants.TOKEN_DIR

In [3]:
#Load data
data_train, labels_train, data_test, labels_test = pickle.load(open(model_data_path,'rb'))
data_train_sample, labels_train_sample, data_test_sample, labels_test_sample = pickle.load(open(sample_data_path,'rb'))

In [4]:
# read in the fitted tokenizer from pickle
with open(tokenizer_path, 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

In [5]:
X_train, X_test, y_train, y_test = data_train_sample, data_test_sample, labels_train_sample, labels_test_sample

In [6]:
#Prep GloVe Embedding
embeddings_index = {}
with open(os.path.join(embedding_path, 'glove.6B.100d.txt'),'r', encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [7]:
# prepare embedding matrix
word_index = tokenizer.word_index
num_words = min(max_word_no, len(word_index) + 1) # Only use the top num_words words from the training corp
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= max_word_no: # if the word index exceeds the limit
        continue # then this word embedding is not included into the embedding
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = tf.keras.layers.Embedding(num_words, # input dim - vocabulary size
                            embedding_dim, # output dim - dense embedding dimension
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_len,
                            trainable=False)

In [8]:
# Initialize parameters and hyper-parameters
weight_vec = list(np.max(np.sum(y_train, axis=0))/np.sum(y_train, axis=0))
class_weight = {i: weight_vec[i] for i in range(y_train.shape[1])}
no_epoch = 20

In [9]:
log_dir = os.path.join('..','log_gender_glove_cnn')
hparam_dir=os.path.join('..','log_gender_glove_cnn','hparam_tuning')

In [15]:
HP_LR = hp.HParam('learning_rate', hp.Discrete([0.0001, 0.001, 0.01]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.1, 0.2, 0.3]))
HP_BATCH = hp.HParam('batch_size', hp.Discrete([128, 256, 512]))
HP_FILTER = hp.HParam('no_filter', hp.Discrete([64, 128]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer(hparam_dir).as_default():
    hp.hparams_config(
        hparams=[HP_LR, HP_DROPOUT, HP_BATCH, HP_FILTER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
      )

In [19]:
def train_test_model(hparams, log_dir):
    # construct the model
    # train a 1D convnet with global maxpooling
    sequence_input = tf.keras.layers.Input(shape=(max_len,), dtype='int32')
    emb = embedding_layer(sequence_input) # turn word index into word embedding

    # Specify each convolution layer and their kernel siz i.e. n-grams 
    conv1_1 = tf.keras.layers.Conv1D(filters=hparams[HP_FILTER], kernel_size=3)(emb)
    btch1_1 = tf.keras.layers.BatchNormalization()(conv1_1)
    drp1_1  = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(btch1_1)
    actv1_1 = tf.keras.layers.Activation('relu')(drp1_1)
    glmp1_1 = tf.keras.layers.GlobalMaxPooling1D()(actv1_1)

    conv1_2 = tf.keras.layers.Conv1D(filters=hparams[HP_FILTER], kernel_size=4)(emb)
    btch1_2 = tf.keras.layers.BatchNormalization()(conv1_2)
    drp1_2  = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(btch1_2)
    actv1_2 = tf.keras.layers.Activation('relu')(drp1_2)
    glmp1_2 = tf.keras.layers.GlobalMaxPooling1D()(actv1_2)

    conv1_3 = tf.keras.layers.Conv1D(filters=hparams[HP_FILTER], kernel_size=5)(emb)
    btch1_3 = tf.keras.layers.BatchNormalization()(conv1_3)
    drp1_3  = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(btch1_3)
    actv1_3 = tf.keras.layers.Activation('relu')(drp1_3)
    glmp1_3 = tf.keras.layers.GlobalMaxPooling1D()(actv1_3)

    conv1_4 = tf.keras.layers.Conv1D(filters=hparams[HP_FILTER], kernel_size=6)(emb)
    btch1_4 = tf.keras.layers.BatchNormalization()(conv1_4)
    drp1_4  = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(btch1_4)
    actv1_4 = tf.keras.layers.Activation('relu')(drp1_4)
    glmp1_4 = tf.keras.layers.GlobalMaxPooling1D()(actv1_4)

    # Gather all convolution layers
    cnct = tf.keras.layers.concatenate([glmp1_1, glmp1_2, glmp1_3, glmp1_4], axis=1)
    drp1 = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(cnct)

    dns1  = tf.keras.layers.Dense(32, activation='relu')(drp1)
    btch1 = tf.keras.layers.BatchNormalization()(dns1)
    drp2  = tf.keras.layers.Dropout(hparams[HP_DROPOUT])(btch1)

    out = tf.keras.layers.Dense(y_train.shape[1], activation='softmax')(drp2)

    model = tf.keras.models.Model(inputs=sequence_input, outputs=out)
    
    adam = Adam(lr=hparams[HP_LR], beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    
    model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(X_train, y_train, validation_split=0.1, epochs=no_epoch, batch_size=hparams[HP_BATCH],
             shuffle=True, class_weight=class_weight, verbose=1,
              callbacks = [tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)]
             )
    _, accuracy = model.evaluate(X_test, y_test)
    return accuracy

In [21]:
session_num = 0

for dropout_rate in HP_DROPOUT.domain.values:
    for learning_rate in HP_LR.domain.values:
        for batch_size in HP_BATCH.domain.values:
            for no_filter in HP_FILTER.domain.values:
                hparams = { HP_DROPOUT: dropout_rate, HP_LR: learning_rate, HP_BATCH: batch_size, HP_FILTER: no_filter }
                run_name = "run-%d" % session_num
                print('--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                logdir = hparam_dir + '\\dp_' + str(hparams[HP_DROPOUT]) + '_lr_' + str(hparams[HP_LR]) + '_bs_' + str(hparams[HP_BATCH]) + '_nf_' + str(hparams[HP_FILTER])
                with tf.summary.create_file_writer(logdir + '\\metrics').as_default():
                    hp.hparams(hparams) # record the values used in this trial
                    accuracy = train_test_model(hparams, logdir)
                    tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
                session_num += 1

--- Starting trial: run-0
{'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 128, 'no_filter': 64}
Train on 40480 samples, validate on 4498 samples
Epoch 1/20
40480/40480 [==============================] - 475s 12ms/sample - loss: 0.8793 - accuracy: 0.5366 - val_loss: 0.7121 - val_accuracy: 0.5609
Epoch 2/20
40480/40480 [==============================] - 471s 12ms/sample - loss: 0.7493 - accuracy: 0.5679 - val_loss: 0.6895 - val_accuracy: 0.5889
Epoch 3/20
40480/40480 [==============================] - 478s 12ms/sample - loss: 0.7172 - accuracy: 0.5850 - val_loss: 0.6854 - val_accuracy: 0.5965
Epoch 4/20
40480/40480 [==============================] - 475s 12ms/sample - loss: 0.7062 - accuracy: 0.5933 - val_loss: 0.6813 - val_accuracy: 0.6103
Epoch 5/20
40480/40480 [==============================] - 478s 12ms/sample - loss: 0.6980 - accuracy: 0.6017 - val_loss: 0.6828 - val_accuracy: 0.5989
Epoch 6/20
40480/40480 [==============================] - 481s 12ms/sample - loss: 0.6908 - 

--- Starting trial: run-1
{'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 128, 'no_filter': 128}
Train on 40480 samples, validate on 4498 samples
Epoch 1/20
40480/40480 [==============================] - 829s 20ms/sample - loss: 0.8222 - accuracy: 0.5527 - val_loss: 0.7416 - val_accuracy: 0.5454
Epoch 2/20
40480/40480 [==============================] - 841s 21ms/sample - loss: 0.7288 - accuracy: 0.5836 - val_loss: 0.6804 - val_accuracy: 0.6189
Epoch 3/20
40480/40480 [==============================] - 832s 21ms/sample - loss: 0.7024 - accuracy: 0.6017 - val_loss: 0.6799 - val_accuracy: 0.6165
Epoch 4/20
40480/40480 [==============================] - 847s 21ms/sample - loss: 0.6910 - accuracy: 0.6153 - val_loss: 0.6707 - val_accuracy: 0.6349
Epoch 5/20
40480/40480 [==============================] - 842s 21ms/sample - loss: 0.6805 - accuracy: 0.6240 - val_loss: 0.6675 - val_accuracy: 0.6374
Epoch 6/20
40480/40480 [==============================] - 841s 21ms/sample - loss: 0.6729 -

--- Starting trial: run-2
{'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 256, 'no_filter': 64}
Train on 40480 samples, validate on 4498 samples
Epoch 1/20
40480/40480 [==============================] - 452s 11ms/sample - loss: 0.9467 - accuracy: 0.5208 - val_loss: 0.7101 - val_accuracy: 0.5518
Epoch 2/20
40480/40480 [==============================] - 444s 11ms/sample - loss: 0.8041 - accuracy: 0.5555 - val_loss: 0.7070 - val_accuracy: 0.5734
Epoch 3/20
40480/40480 [==============================] - 442s 11ms/sample - loss: 0.7549 - accuracy: 0.5691 - val_loss: 0.6983 - val_accuracy: 0.5860
Epoch 4/20
40480/40480 [==============================] - 440s 11ms/sample - loss: 0.7368 - accuracy: 0.5750 - val_loss: 0.6869 - val_accuracy: 0.6063
Epoch 5/20
40480/40480 [==============================] - 443s 11ms/sample - loss: 0.7181 - accuracy: 0.5886 - val_loss: 0.6804 - val_accuracy: 0.6158
Epoch 6/20
40480/40480 [==============================] - 443s 11ms/sample - loss: 0.7109 - 

--- Starting trial: run-3
{'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 256, 'no_filter': 128}
Train on 40480 samples, validate on 4498 samples
Epoch 1/20
40480/40480 [==============================] - 824s 20ms/sample - loss: 0.9025 - accuracy: 0.5299 - val_loss: 0.7637 - val_accuracy: 0.5018
Epoch 2/20
40480/40480 [==============================] - 811s 20ms/sample - loss: 0.7739 - accuracy: 0.5671 - val_loss: 0.7219 - val_accuracy: 0.5462
Epoch 3/20
40480/40480 [==============================] - 801s 20ms/sample - loss: 0.7361 - accuracy: 0.5811 - val_loss: 0.7066 - val_accuracy: 0.5729
Epoch 4/20
40480/40480 [==============================] - 809s 20ms/sample - loss: 0.7132 - accuracy: 0.5956 - val_loss: 0.7096 - val_accuracy: 0.5683
Epoch 5/20
40480/40480 [==============================] - 816s 20ms/sample - loss: 0.6989 - accuracy: 0.6041 - val_loss: 0.6833 - val_accuracy: 0.6023
Epoch 6/20
40480/40480 [==============================] - 813s 20ms/sample - loss: 0.6870 -

--- Starting trial: run-4
{'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 512, 'no_filter': 64}
Train on 40480 samples, validate on 4498 samples
Epoch 1/20
40480/40480 [==============================] - 471s 12ms/sample - loss: 0.9156 - accuracy: 0.5246 - val_loss: 0.7097 - val_accuracy: 0.5538
Epoch 2/20
40480/40480 [==============================] - 447s 11ms/sample - loss: 0.8014 - accuracy: 0.5466 - val_loss: 0.7073 - val_accuracy: 0.5569
Epoch 3/20
40480/40480 [==============================] - 446s 11ms/sample - loss: 0.7659 - accuracy: 0.5621 - val_loss: 0.6973 - val_accuracy: 0.5765
Epoch 4/20
40480/40480 [==============================] - 447s 11ms/sample - loss: 0.7430 - accuracy: 0.5687 - val_loss: 0.6936 - val_accuracy: 0.5863
Epoch 5/20
40480/40480 [==============================] - 444s 11ms/sample - loss: 0.7282 - accuracy: 0.5789 - val_loss: 0.6893 - val_accuracy: 0.5963
Epoch 6/20
40480/40480 [==============================] - 444s 11ms/sample - loss: 0.7215 - 

--- Starting trial: run-5
{'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 512, 'no_filter': 128}
Train on 40480 samples, validate on 4498 samples
Epoch 1/20
40480/40480 [==============================] - 705s 17ms/sample - loss: 0.8495 - accuracy: 0.5327 - val_loss: 0.7036 - val_accuracy: 0.5676
Epoch 2/20
40480/40480 [==============================] - 688s 17ms/sample - loss: 0.7558 - accuracy: 0.5633 - val_loss: 0.7010 - val_accuracy: 0.5629
Epoch 3/20
40480/40480 [==============================] - 692s 17ms/sample - loss: 0.7290 - accuracy: 0.5762 - val_loss: 0.7008 - val_accuracy: 0.5571
Epoch 4/20
40480/40480 [==============================] - 706s 17ms/sample - loss: 0.7112 - accuracy: 0.5917 - val_loss: 0.6941 - val_accuracy: 0.5760
Epoch 5/20
40480/40480 [==============================] - 708s 17ms/sample - loss: 0.7049 - accuracy: 0.5931 - val_loss: 0.6848 - val_accuracy: 0.5925
Epoch 6/20
40480/40480 [==============================] - 706s 17ms/sample - loss: 0.6935 -

KeyboardInterrupt: 